In [130]:
from itertools import product
import numpy as np
from tqdm import tqdm
from mdptoolbox import mdp
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
%matplotlib inline

In [131]:
# game (2 players) settings,
# hi - lo = 1 => reduce to IPD; lo~defect, hi~coop
lo, hi = 0, 1

def r1(a1, a2):
    return min(a1, a2) + 2 * np.sign(a2 - a1)

np.random.seed(618)
k_max = 1

print([(r1(0, a2), r1(a2, 0)) for a2 in range(lo, hi + 1)])
print([(r1(1, a2), r1(a2, 1)) for a2 in range(lo, hi + 1)])

[(0, 0), (2, -2)]
[(-2, 2), (1, 1)]


In [132]:
actions = list(range(lo, hi + 1))
hists = []
for k in range(k_max + 1):
    h_k = list(map(list, product(actions, repeat=k * 2)))
    hists += h_k
hists

[[], [0, 0], [0, 1], [1, 0], [1, 1]]

In [133]:
# specify the prob of playing (defect, coop)
pi_D = ([1, 0], [1, 0], [1, 0], [1, 0], [1, 0]) # always-defect
pi_C = ([0, 1], [0, 1], [0, 1], [0, 1], [0, 1]) # always-coop
pi_TFT = ([0, 1], [1, 0], [1, 0], [0, 1], [0, 1]) # Tit-For-Tat from coop
pi_TFT_D = ([1, 0], [1, 0], [1, 0], [0, 1], [0, 1]) # Tit-For-Tat from coop
pi_syn= ([0.715, 0.285], [0.715, 0.285], [0.715, 0.285], [0.715, 0.285], [0.715, 0.285]) # a syn-ed one

In [134]:
num_actions = len(actions)
num_hists = len(hists)

def comp_R(pi_2):
    R = np.zeros(shape=(num_hists, num_actions))
    for i, h in enumerate(hists):
        for k1, a1 in enumerate(actions):
            for k2, a2 in enumerate(actions):
                R[i, k1] += r1(a1, a2) * pi_2[i][k2]
    return R

def comp_T(pi_2):
    T = np.zeros(shape=(num_actions, num_hists, num_hists))
    for k1, a1 in enumerate(actions):
        for i, hi in enumerate(hists):
            for k2, a2 in enumerate(actions):
                hj = hi[2:] + [a1, a2]
                # avoid using list.index()
                l = len(hj)
                base = [num_actions ** i for i in range(l - 1, -1, -1)]
                j = np.dot(np.array(hj), base) + sum([num_actions ** m for m in range(l - 2, -1 , -2)])
                # print(hj, hists.index(hj), j)
                T[k1, i, j] += pi_2[i][k2]
    T = T / np.sum(T, axis=2, keepdims=True)
    return T
print(comp_R(pi_TFT))
print(comp_T(pi_TFT))

[[ 2.  1.]
 [ 0. -2.]
 [ 0. -2.]
 [ 2.  1.]
 [ 2.  1.]]
[[[0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 1.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1.]]]


In [135]:
def comp_BR(pi_2):
    solver = mdp.PolicyIteration(transitions=comp_T(pi_2), reward=comp_R(pi_2), discount=0.99)
    solver.run()
    return solver.policy, solver.V

In [138]:
comp_BR(pi_C)

((0, 0, 0, 0, 0),
 (199.99999999999983,
  199.99999999999983,
  199.99999999999983,
  199.99999999999983,
  199.99999999999983))